# Scraping

this code provide how to scrap the [D&D 5e Wiki](https://dnd-5e.fandom.com/wiki/D%26D_5e_Wiki)

In [1]:
import requests
import re
import os
import pandas as pd
import markdownify 

from io import StringIO
from bs4 import BeautifulSoup, Comment
from tqdm import tqdm
from IPython.display import display

In [2]:
url = requests.get('https://dnd-5e.fandom.com/wiki/List_of_Spells')
soup = BeautifulSoup(url.content, 'html.parser')

# Spell

filter out `Unearthed Arcana`

In [3]:
data = soup.find_all('span', 'mw-headline')
filtered_data = [item for item in data if item.text != 'Unearthed Arcana' and item.text != 'Spell Navigation']

In [4]:
filtered_data

[<span class="mw-headline" id="Cantrips">Cantrips</span>,
 <span class="mw-headline" id="1st_Level">1st Level</span>,
 <span class="mw-headline" id="2nd_Level">2nd Level</span>,
 <span class="mw-headline" id="3rd_Level">3rd Level</span>,
 <span class="mw-headline" id="4th_Level">4th Level</span>,
 <span class="mw-headline" id="5th_Level">5th Level</span>,
 <span class="mw-headline" id="6th_Level">6th Level</span>,
 <span class="mw-headline" id="7th_Level">7th Level</span>,
 <span class="mw-headline" id="8th_Level">8th Level</span>,
 <span class="mw-headline" id="9th_Level">9th Level</span>]

In [5]:
dfs = []

for table in filtered_data:
    table = table.find_next('table')
    html_table = list()
    
    if table:
        html_string = str(table)
        html_table.append(table)
        df = pd.read_html(StringIO(html_string))[0]
        dfs.append(df)

print(len(dfs))
display(dfs[0].head())
display(dfs[1].head())

10


,Name,School,Casting Time,Comps,Save,Conc.,Source
0,Acid Splash,Conjuration,1 action,"V, S",Dexterity,NaN,PHB 211
1,Blade Ward,Abjuration,1 action,"V, S",NaN,NaN,PHB 218
2,Booming Blade,Evocation,1 action,"V, M",Attack roll,NaN,"TCE 143, SCAG 142"
3,Chill Touch,Necromancy,1 action,"V, S",Attack roll,NaN,PHB 221
4,Control Flames,Transmutation,1 action,S,NaN,NaN,"XGE 152, EEPC 16"


,Name,School,Casting Time,Comps,Cost,Save,Ritual,Conc.,Source
0,Absorb Elements,Abjuration,1 reaction,S,NaN,NaN,NaN,NaN,"XGE 150, EEPC 15"
1,Alarm,Abjuration,1 minute,"V, S, M",NaN,NaN,Yes,NaN,PHB 211
2,Animal Friendship,Enchantment,1 action,"V, S, M",NaN,Wisdom,NaN,NaN,PHB 212
3,Armor of Agathys,Abjuration,1 action,"V, S, M",NaN,NaN,NaN,NaN,PHB 215
4,Arms of Hadar,Conjuration,1 action,"V, S",NaN,Strength,NaN,NaN,PHB 215


In [6]:
os.makedirs("../datasets/spell/", exist_ok=True)

for title, df in zip(filtered_data, dfs):
    df.to_csv(f"../datasets/spell/{title.text}.csv", index=False)

In [7]:
filtered_data

[<span class="mw-headline" id="Cantrips">Cantrips</span>,
 <span class="mw-headline" id="1st_Level">1st Level</span>,
 <span class="mw-headline" id="2nd_Level">2nd Level</span>,
 <span class="mw-headline" id="3rd_Level">3rd Level</span>,
 <span class="mw-headline" id="4th_Level">4th Level</span>,
 <span class="mw-headline" id="5th_Level">5th Level</span>,
 <span class="mw-headline" id="6th_Level">6th Level</span>,
 <span class="mw-headline" id="7th_Level">7th Level</span>,
 <span class="mw-headline" id="8th_Level">8th Level</span>,
 <span class="mw-headline" id="9th_Level">9th Level</span>]

In [8]:
data = soup.find_all('span', 'mw-headline')
filtered_data = [item for item in data if item.text != 'Unearthed Arcana' and item.text != 'Spell Navigation']  
os.makedirs("../datasets/spell_content", exist_ok=True)

for table in filtered_data:
    file_name = table.text
    table = table.find_next('table')
    data = list()

    if table:
        anchors = table.find_all('a')
        
        for anchor in tqdm(anchors):
            url = requests.get(f'https://dnd-5e.fandom.com{anchor['href']}')
            soup = BeautifulSoup(url.content, 'html.parser')
            name_tag = soup.find('span', class_='mw-page-title-main')

            h2_tags = soup.find_all('h2')

            for h2_tag in h2_tags:
                headline = h2_tag.find('span', class_='mw-headline')
                if headline:
                    new_h2 = soup.new_tag("h2")
                    new_h2.string = headline.text.strip()
                    h2_tag.replace_with(new_h2)

            content_block = soup.find('div', 'mw-content-ltr mw-parser-output')
            content_children = content_block.findChildren()
            cleaned_content = []

            for element in content_children:
                markdown = markdownify.markdownify(str(element), strip=['a', 'span'], heading_style="ATX")
                cleaned_content.append(markdown)

            combined_content = f"# {name_tag.text.strip()}\n" + "\n".join(cleaned_content) + "\n"
            combined_content = re.sub(r'<!--.*?-->', '', combined_content, flags=re.DOTALL)

            data.append(combined_content)

        with open(f"../datasets/spell_content/{file_name}.txt", "w", encoding="utf-8") as f:
            for item in data:
                f.write(f"{item}\n")

  0%|          | 0/46 [00:00<?, ?it/s]/tmp/ipykernel_271395/831663796.py:28: DeprecationWarning: Call to deprecated method findChildren. (Replaced by find_all) -- Deprecated since version 3.0.0.
  content_children = content_block.findChildren()
100%|██████████| 22/22 [00:08<00:00,  2.66it/s]


In [9]:
url = requests.get('https://dnd-5e.fandom.com/wiki/Acid_Splash')
soup = BeautifulSoup(url.content, 'html.parser')

In [10]:
soup.find('h1', 'page-header__title')

<h1 class="page-header__title" id="firstHeading">
<span class="mw-page-title-main">Acid Splash</span> </h1>

In [11]:
soup.find('div', 'mw-content-ltr mw-parser-output')

<div class="mw-content-ltr mw-parser-output" dir="ltr" lang="en"><p>From Player's Handbook, page 211.
</p>
<h2><span class="mw-headline" id="Description">Description</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a class="mw-editsection-visualeditor" data-action="edit-section" data-testid="log-in-edit-section" data-tracking-label="log-in-edit-section" href="https://auth.fandom.com/signin?redirect=https%3A%2F%2Fdnd-5e.fandom.com%2Fwiki%2FAcid_Splash%3Fveaction%3Dedit%26section%3D1&amp;uselang=en" title="Sign in to edit"><svg class="wds-icon wds-icon-tiny"><use xlink:href="#wds-icons-pencil-tiny"></use></svg></a><span class="mw-editsection-bracket">]</span></span></h2>
<p><i>Conjuration cantrip</i>
</p>
<ul><li><b>Casting Time:</b> 1 action</li>
<li><b>Range:</b> 60 feet</li>
<li><b>Components:</b> V, S</li>
<li><b>Duration:</b> Instantaneous</li></ul>
<p>You hurl a bubble of acid. Choose one creature you can see within range, or choose two creatures you

In [12]:
from bs4 import BeautifulSoup

html = '''
<h2><span class="mw-headline" id="Learned_By">Learned By</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a class="mw-editsection-visualeditor" data-action="edit-section" data-testid="log-in-edit-section" data-tracking-label="log-in-edit-section" href="https://auth.fandom.com/signin?redirect=https%3A%2F%2Fdnd-5e.fandom.com%2Fwiki%2FAcid_Splash%3Fveaction%3Dedit%26section%3D2&amp;uselang=en" title="Sign in to edit"><svg class="wds-icon wds-icon-tiny"><use xlink:href="#wds-icons-pencil-tiny"></use></svg></a><span class="mw-editsection-bracket">]</span></span></h2>
'''

soup = BeautifulSoup(html, 'html.parser')

h2 = soup.find('h2')
headline_text = h2.find('span', class_='mw-headline').text.strip()

# Create a clean new <h2> tag
print(headline_text)
new_h2 = soup.new_tag("h2")
new_h2.string = headline_text

print(new_h2)

Learned By
<h2>Learned By</h2>
